In [1]:
# installing the required libraries
!pip install numpy
!pip install pandas
!pip install nltk

In [2]:
# importing the necessary libraries
import numpy as npy
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [61]:
# loading the data
df=pd.read_csv('D:\DBDA\Projects\Flipkart_Review_Analysis\data.csv')
df.head()

,review,rating
0,It was nice produt. I like it's design a lot. ...,5
1,awesome sound....very pretty to see this nd th...,5
2,awesome sound quality. pros 7-8 hrs of battery...,4
3,I think it is such a good product not only as ...,5
4,awesome bass sound quality very good bettary l...,5


In [5]:
# Performing EDA

# Dimensions of the data
df.shape

(9976, 2)

In [7]:
# Getting column names
df.columns

Index(['review', 'rating'], dtype='object')

In [62]:
# removing the duplicates from the data as duplicates can lead to bias in the data
df.drop_duplicates(inplace=True)
df.shape

(7868, 2)

In [63]:
# check if there are any missing values in the data
df.isnull().sum()

review    0
rating    0
dtype: int64

In [64]:
# modifying the data frame replacing the rating with positive or negative
df.replace(to_replace=[1,2,3],value=0,inplace=True)
df.replace(to_replace=[4,5],value=1,inplace=True)
df

,review,rating
0,It was nice produt. I like it's design a lot. ...,1
1,awesome sound....very pretty to see this nd th...,1
2,awesome sound quality. pros 7-8 hrs of battery...,1
3,I think it is such a good product not only as ...,1
4,awesome bass sound quality very good bettary l...,1
...,...,...
9965,good headphoneREAD MORE,1
9967,Bad product.READ MORE,0
9968,It's tight and not more comfortableREAD MORE,1
9972,Everything is amazimg but the built is very li...,1


In [12]:
# downloading the stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shreyas\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [15]:
# viewing the stop words
print([stopwords.words('english')])

[['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 

In [37]:
# Data cleaning
    # 1. removing the punctuation marks
    # 2. removing the stop words
def cleanData(text):
    # removing the punctuation
    punc_marks = [char for char in text if char not in string.punctuation]
    no_punc=''.join(punc_marks)
    
    # removing the stopwords
    cleaned_data=[word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
    return cleaned_data

In [38]:
cleanData("Hello! Shreyas, How are You? Have You met me?")

['Hello', 'Shreyas', 'met']

In [40]:
# checking if the function is working or not
df['review'].head().apply(cleanData)

0    [nice, produt, like, design, lot, easy, carry,...
1    [awesome, soundvery, pretty, see, nd, sound, q...
2    [awesome, sound, quality, pros, 78, hrs, batte...
3    [think, good, product, per, quality, also, des...
4    [awesome, bass, sound, quality, good, bettary,...
Name: review, dtype: object

In [65]:
# converting the text data into matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
tkns=CountVectorizer(analyzer=cleanData).fit_transform(df['review'])
# print(tkns)

In [66]:
# Splitting the data into test(20%) and train(80%) data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(tkns, df['rating'], test_size = 0.2, random_state = 0)

In [67]:
tkns.shape

(7868, 12365)

In [68]:
# Creating and training the Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB().fit(X_train,y_train)

In [82]:
# Predicting the model on the test data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
model = classifier.predict(X_test)
print(f"Report: \n{classification_report(y_test,model)}")
print(f"Confusion Matrix: \n{confusion_matrix(y_test,model)}\n")
print(f"Accuracy: {accuracy_score(y_test,model)*100}")

Report: 
              precision    recall  f1-score   support

           0       0.68      0.53      0.60       326
           1       0.88      0.94      0.91      1248

    accuracy                           0.85      1574
   macro avg       0.78      0.73      0.75      1574
weighted avg       0.84      0.85      0.84      1574

Confusion Matrix: 
[[ 172  154]
 [  80 1168]]

Accuracy: 85.13341804320204
